# Brute Force: 100 Dimensions (Ackley Function)

This notebook demonstrates how to use the `Spot` class from `spotpython` with and without the Nyström approximation for Kriging surrogates on the 100-dimensional Ackley function.

We use a maximum of 500 function evaluations.

In [1]:
import numpy as np
from scipy.optimize import minimize
import time
import warnings

# ---
# 1. Definition der Zielfunktion: 100D Ackley
# ---
# Basierend auf der Standarddefinition [1, 2, 3]
# Globales Optimum: f(x*) = 0 bei x* = [0,..., 0]
# ---

def ackley(x):
    """
    Berechnet die n-dimensionale Ackley-Funktion.
    Parameter a=20, b=0.2, c=2*pi werden standardmäßig verwendet.
    """
    a = 20.0
    b = 0.2
    c = 2.0 * np.pi
    
    # Dimension aus dem Input-Vektor ableiten
    d = len(x)  
    
    # Berechne die beiden Terme der Funktion
    sum_sq_term = -b * np.sqrt(np.sum(x**2) / d)
    cos_term = np.sum(np.cos(c * x)) / d
    
    # Kombiniere die Terme
    result = -a * np.exp(sum_sq_term) - np.exp(cos_term) + a + np.e
    
    return result

# ---
# 2. Definition der globalen Parameter und der Budget-Allokation
# ---

# Problem-Dimension
DIMENSIONS = 100

# Suchraum-Grenzen 
BOUND_LOW = -32.768
BOUND_HIGH = 32.768

# Globales Budget für Funktionsauswertungen
BUDGET_TOTAL = 500

# ---
# 3. Strategische Bestimmung von n (Anzahl Starts) und b (Budget pro Lauf)
# ---
# Wie in Abschnitt 2 des Berichts hergeleitet:
# Wir wählen n=10, um eine minimale Exploration zu gewährleisten.
# Dies lässt b=50 FEs pro lokaler Suche (inkl. Start-FE).
# n * b = 10 * 50 = 500.
# ---

N_STARTS = 25
BUDGET_PER_RUN = BUDGET_TOTAL // N_STARTS  # Ergibt 50

# ---
# 4. Haupt-Optimierungsfunktion
# ---

def run_multi_start_optimization():
    """
    Führt die Multi-Start-Optimierung mit dem Powell-Verfahren durch.
    """
    print("--- Expertenbericht: Multi-Start-Optimierung (100D Ackley) ---")
    print(f"Dimension (D):         {DIMENSIONS}")
    print(f"Gesamtbudget (FEs):    {BUDGET_TOTAL}")
    print(f"Lokaler Suchalgorithmus: 'Powell' (ableitungsfrei)")
    print("---")
    print(f"Strategische Budget-Allokation:")
    print(f"  Anzahl Starts (n):   {N_STARTS}")
    print(f"  Budget pro Start (b): {BUDGET_PER_RUN} FEs (lokale Suche)")
    print(f"  Erwartete Gesamt-FEs: {N_STARTS * BUDGET_PER_RUN}")
    print("---")
    
    start_time = time.time()
    
    # Für Reproduzierbarkeit
    rng = np.random.default_rng(seed=42)
    
    # Generiere n Startpunkte [13, 14]
    # Shape: (N_STARTS, DIMENSIONS)
    start_points = rng.uniform(BOUND_LOW, BOUND_HIGH, size=(N_STARTS, DIMENSIONS))
    
    # Definiere die Bounds für scipy.optimize.minimize [4, 10]
    # Powell unterstützt 'bounds'
    # bounds = * DIMENSIONS
    bounds = [(BOUND_LOW, BOUND_HIGH)] * DIMENSIONS
    
    # Optionen für den Powell-Minimierer 
    # Wir setzen maxfev auf unser berechnetes lokales Budget b.
    powell_options = {
        'maxfev': BUDGET_PER_RUN,
        'disp': False  # Ausgabe pro Lauf unterdrücken
    }
    
    # Speichern der Ergebnisse
    all_results = []
    total_fevals_used = 0
    
    print("Starte Optimierungsläufe...")
    
    # Unterdrücke Warnungen von SciPy, falls maxfev erreicht wird
    warnings.filterwarnings("ignore", message=".*Maximum number of function evaluations.*")
    
    # --- Die Hauptschleife ---
    for i in range(N_STARTS):
        x0 = start_points[i]
        
        # Starte die lokale Suche
        # Die erste Auswertung (f(x0)) wird von minimize durchgeführt
        # und zählt zum Budget 'maxfev'.
        result = minimize(
            fun=ackley,
            x0=x0,
            method='Powell',
            bounds=bounds,
            options=powell_options
        )
        
        # Speichere relevante Daten
        all_results.append({
            'run_id': i,
            'f_start': ackley(x0) if result.nfev == 0 else np.nan, # Fallback
            'f_end': result.fun,
            'nfev_local': result.nfev,  # FEs, die minimize genutzt hat
            'success': result.success,
            'message': result.message,
            'best_x': result.x
        })
        
        total_fevals_used += result.nfev
        
        # Der Startwert f(x0) ist die erste Auswertung.
        # Wir müssen ihn separat holen, falls minimize ihn nicht liefert (sollte es aber)
        # Für das Reporting holen wir den Startwert:
        f_start_val = all_results[i]['f_start']
        if np.isnan(f_start_val) and result.nfev > 0:
             # Normalerweise nicht nötig, aber zur Sicherheit
             f_start_val = ackley(x0)
             # Diese FE ist bereits in result.nfev enthalten
        
        print(f"  Lauf {i+1}/{N_STARTS} abgeschlossen. "
              f"End-Wert: {result.fun:.4f} "
              f"(Lokale FEs: {result.nfev}/{BUDGET_PER_RUN})")
        
    end_time = time.time()
    
    # ---
    # 5. Ergebnisanalyse und Zusammenfassung
    # ---
    print("---")
    print("Alle Optimierungsläufe abgeschlossen.")
    print(f"Gesamtlaufzeit: {end_time - start_time:.2f} Sekunden")
    print(f"Verbrauchtes Gesamtbudget: {total_fevals_used} / {BUDGET_TOTAL} FEs")
    print("---")
    
    # Finde das beste Ergebnis über alle Läufe
    best_run = min(all_results, key=lambda r: r['f_end'])
    
    print("\n--- Zusammenfassung der Ergebnisse ---")
    
    # Erzeuge die Ergebnistabelle
    print("Tabelle 1: Ergebnisse der Multi-Start-Optimierungsläufe")
    print("=" * 70)
    print(f"{'Lauf-ID':<8} | {'f(x_end)':<12} | {'Lokale FEs':<10} | {'Status'}")
    print("-" * 70)
    for res in all_results:
        # Status-Nachricht kürzen
        status = "Max FEs erreicht" if "function evaluations" in res['message'] else "Andere"
        if res['success']: status = "Konvergiert"
            
        print(f"{res['run_id']:<8} | {res['f_end']:<12.4f} | {res['nfev_local']:<10} | {status}")
    print("=" * 70)
    
    print("\n--- Bestes gefundenes Minimum ---")
    print(f"Bester Lauf (ID):     {best_run['run_id']}")
    print(f"Bester Funktionswert: {best_run['f_end']:.8f}")
    print(f"  (Globales Optimum ist: 0.0)")
    # Zeige die ersten 5 von 100 Dimensionen des besten Vektors
    x_best = best_run['best_x']
    print(f"  Bester Vektor (x*):   [{x_best[0]:.2f}, {x_best[1]:.2f}, {x_best[2]:.2f}, {x_best[3]:.2f}, {x_best[4]:.2f}, ...]")
    
    return best_run

# ---
# 6. Skriptausführung
# ---
if __name__ == "__main__":
    best_result = run_multi_start_optimization()

--- Expertenbericht: Multi-Start-Optimierung (100D Ackley) ---
Dimension (D):         100
Gesamtbudget (FEs):    500
Lokaler Suchalgorithmus: 'Powell' (ableitungsfrei)
---
Strategische Budget-Allokation:
  Anzahl Starts (n):   25
  Budget pro Start (b): 20 FEs (lokale Suche)
  Erwartete Gesamt-FEs: 500
---
Starte Optimierungsläufe...
  Lauf 1/25 abgeschlossen. End-Wert: 21.1100 (Lokale FEs: 20/20)
  Lauf 2/25 abgeschlossen. End-Wert: 21.3363 (Lokale FEs: 20/20)
  Lauf 3/25 abgeschlossen. End-Wert: 21.3037 (Lokale FEs: 20/20)
  Lauf 4/25 abgeschlossen. End-Wert: 21.1079 (Lokale FEs: 20/20)
  Lauf 5/25 abgeschlossen. End-Wert: 21.1091 (Lokale FEs: 20/20)
  Lauf 6/25 abgeschlossen. End-Wert: 21.2609 (Lokale FEs: 20/20)
  Lauf 7/25 abgeschlossen. End-Wert: 21.2182 (Lokale FEs: 20/20)
  Lauf 8/25 abgeschlossen. End-Wert: 21.3431 (Lokale FEs: 20/20)
  Lauf 9/25 abgeschlossen. End-Wert: 21.2842 (Lokale FEs: 20/20)
  Lauf 10/25 abgeschlossen. End-Wert: 21.3092 (Lokale FEs: 20/20)
  Lauf 11/25 